In [175]:
import pickle
import sqlite3

In [177]:
database_path=r"..\dataset\covid-19\mysql_database\covid19.db"
parsed_parameter_save_path='../dataset/covid-19/parsed_parameters.pickle'
print_sql_queries=True

In [178]:
with open(parsed_parameter_save_path, 'rb') as f:
    queries, state_country_dict, place_lower_to_normal=pickle.load(f)
# queries

In [179]:
con = sqlite3.connect(database_path)

In [180]:
case_type_dict={
    'confirm': 'Confirmed',
    'recover': 'Recovered',
    'death': 'Deaths',
    'increase rate': 'Increase_rate',
    'active': 'Active'
}

func_type_dict={
    'maximum': 'MAX',
    'minimum': 'MIN',
    'average': 'AVG',
    'sum': ''
}

In [181]:
def is_us_state(state):
    if state_country_dict[state]=='us':
        return True
    return False

# def extra_result(table, state, time, country, print_sql):
#     print("Other Details:")
#     if table=='us':


def generate_case_query_result(query,print_sql):
    case_type=case_type_dict[query['Case Type']]
    func_type=func_type_dict[query['Function Type']]
    time=query['Time Duration']

    if query['Place']['states']:
        state=place_lower_to_normal[query['Place']['states'][0]]
        if is_us_state(query['Place']['states'][0]):
            if case_type == 'Active' or case_type == 'Recovered' or case_type == 'Increase_rate':
                # print(case_type+" Data is not provided for this State. Other related information about "+state+" is given below:")
                print(case_type+" Data is not provided for this State.")
                # extra_result(table='us',state=state,time=time,country='us',print_sql=print_sql)
                return

            else:
                if func_type=='':
                    # sql = "SELECT "+case_type+" FROM us WHERE Province_State = '"+state+ "' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"';"
                    sql = "SELECT SUM("+case_type+") FROM us WHERE Province_State = '"+state+ "' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Province_State;"
                    if print_sql:
                        print(sql)
                    cur = con.execute(sql)
                    a = cur.fetchall()
                    ans = a[-1][0]-a[0][0]
                    print(ans)
                    return

                else:
                    sql="SELECT "+func_type+"(sum) from"+"(SELECT SUM("+case_type+") as sum FROM us WHERE Province_State = '"+state+ "' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Province_State);"
                    if print_sql:
                        print(sql)
                    cur = con.execute(sql)
                    ans = cur.fetchone()
                    print(ans[0])
                    return
        
        else:
            if case_type == 'Increase_rate':
                # print(case_type+" Data is not provided for this State. Other related information about "+state+" is given below:")
                # extra_result(table='timeseries', state=state,time=time,country=state_country_dict[query['Place']['states'][0]],print_sql=print_sql)
                print(case_type+" Data is not provided for this State.")
                return
            
            elif case_type == 'Active':
                case_type = "Confirmed-Recovered-Deaths"
            
            if func_type=='':
                sql="SELECT "+case_type+" FROM timeseries WHERE Province_State = '"+state+ "' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"';"
                if print_sql:
                    print(sql)
                cur = con.execute(sql)
                a = cur.fetchall()
                ans = a[-1][0]-a[0][0]
                print(ans)
                return

            else:
                sql="SELECT "+func_type+"("+case_type+") FROM timeseries WHERE Province_State = '"+state+ "' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"';"
                if print_sql:
                    print(sql)
                cur = con.execute(sql)
                ans = cur.fetchone()
                print(ans[0])
                return

    elif query['Place']['countries']:
        country=place_lower_to_normal[query['Place']['countries'][0]]
        if case_type == 'Increase_rate':
            # print(case_type+" Data is not provided for this Country. Other related information about "+country+" is given below:")
            # extra_result(table='countries_aggregated', state=None,time=time,country=country,print_sql=print_sql)
            print(case_type+" Data is not provided for "+country+".")
            return
        
        elif case_type == 'Active':
            case_type = "Confirmed-Recovered-Deaths"
        
        if func_type=='':
            sql="SELECT "+case_type+" FROM countries_aggregated WHERE Country = '"+country+ "' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"';"
            if print_sql:
                print(sql)
            cur = con.execute(sql)
            a = cur.fetchall()
            ans = a[-1][0]-a[0][0]
            print(ans)
            return

        else:
            sql="SELECT "+func_type+"("+case_type+") FROM countries_aggregated WHERE Country = '"+country+ "' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"';"
            if print_sql:
                print(sql)
            cur = con.execute(sql)
            ans = cur.fetchone()
            print(ans[0])
            return
    
    else:
        if case_type == 'Active':
            case_type = "Confirmed-Recovered-Deaths"
        
        if case_type == 'Increase_rate':
            time['begin']=max(time['begin'],'2020-01-23')
        
        if func_type=='':
            sql="SELECT "+case_type+" FROM worldwide_aggregate WHERE Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"';"
            if print_sql:
                print(sql)
            cur = con.execute(sql)
            a = cur.fetchall()
            ans = a[-1][0]-a[0][0]
            print(ans)
            return

        else:
            sql="SELECT "+func_type+"("+case_type+") FROM worldwide_aggregate WHERE Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"';"
            if print_sql:
                print(sql)
            cur = con.execute(sql)
            ans = cur.fetchone()
            print(ans[0])
            return

def generate_place_query_result(query,print_sql):
    case_type=case_type_dict[query['Case Type']]
    func_type=func_type_dict[query['Function Type']]
    time=query['Time Duration']

    if query['Operation Type']=='country' and query['Place']['countries']:
        query['Operation Type']='state'
    
    if query['Operation Type']=='state':
        if query['Place']['countries']:
            if 'us' in query['Place']['countries']:
                if case_type == 'Active' or case_type == 'Recovered' or case_type == 'Increase_rate':
                    # print(case_type+" Data is not provided for US. Other related information about US is given below:")
                    # extra_result(table='countries_aggregated',state=None,time=time,country='us',print_sql=print_sql)
                    print(case_type+" Data is not provided for US.")
                    return

                else:
                    if func_type=='':
                        func_type='MAX'

                    sql = "SELECT Province_State FROM (SELECT Province_State, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM("+case_type+") as sum FROM us WHERE Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Province_State) GROUP BY Province_State) WHERE cases = (SELECT "+func_type+"(cases) from (SELECT (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM("+case_type+") as sum FROM us WHERE Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Province_State) GROUP BY Province_State));"
                    
                    if print_sql:
                        print(sql)
                    cur = con.execute(sql)
                    ans = cur.fetchone()
                    print(ans[0])
                    return
            else:
                if case_type == 'Increase_rate':
                    country=place_lower_to_normal[query['Place']['countries'][0]] 
                    # print(case_type+" Data is not provided for"+country+". Other related information about "+country+" is given below:")
                    # extra_result(table='countries_aggregated',state=None,time=time,country=query['Place']['countries'][0],print_sql=print_sql)
                    print(case_type+" Data is not provided for "+country+".")
                    return

                else:
                    if func_type=='':
                        func_type='MAX'
                    
                    if case_type == 'Active':
                        case_type = "Confirmed-Recovered-Deaths"
                    
                    country=place_lower_to_normal[query['Place']['countries'][0]] 
                    sql = "SELECT Province_State FROM (SELECT Province_State, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM("+case_type+") as sum FROM timeseries WHERE Country_Region = '"+country+"' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Province_State) GROUP BY Province_State) WHERE cases = (SELECT "+func_type+"(cases) from (SELECT Province_State, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM("+case_type+") as sum FROM timeseries WHERE Country_Region = '"+country+"' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Province_State) GROUP BY Province_State));"
                    
                    if print_sql:
                        print(sql)
                    cur = con.execute(sql)
                    ans = cur.fetchone()
                    print(ans[0])
                    return
        else:
            if case_type == 'Increase_rate':
                # print(case_type+" Data is not provided for States. Other related information about world is given below:")
                # extra_result(table='worldwide_aggregate',state=None,time=time,country=None,print_sql=print_sql)
                print(case_type+" Data is not provided for States.")
                return

            else:
                if func_type=='':
                    func_type='MAX'
                
                if case_type == 'Active':
                    case_type = "Confirmed-Recovered-Deaths"
                
                sql = "SELECT Province_State FROM (SELECT Province_State, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM("+case_type+") as sum FROM timeseries WHERE Province_State <> '' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Province_State) GROUP BY Province_State) WHERE cases = (SELECT "+func_type+"(cases) from (SELECT Province_State, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM("+case_type+") as sum FROM timeseries WHERE Province_State <> '' AND Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Province_State) GROUP BY Province_State));"
                
                if print_sql:
                    print(sql)
                cur = con.execute(sql)
                ans = cur.fetchone()
                print(ans[0])
                return
    elif query['Operation Type']=='country':
        if case_type == 'Increase_rate':
            # print(case_type+" Data is not provided for Countries. Other related information about World is given below:")
            # extra_result(table='worldwide_aggregate',state=None,time=time,country=None,print_sql=print_sql)
            print(case_type+" Data is not provided for Countries.")
            return

        else:
            if func_type=='':
                func_type='MAX'
            
            if case_type == 'Active':
                case_type = "Confirmed-Recovered-Deaths"
            
            sql = "SELECT Country FROM (SELECT Country, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Country, SUM("+case_type+") as sum FROM countries_aggregated WHERE Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Country) GROUP BY Country) WHERE cases = (SELECT "+func_type+"(cases) from (SELECT Country, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Country, SUM("+case_type+") as sum FROM countries_aggregated WHERE Date BETWEEN '"+time['begin']+"' AND '"+time['end']+"' GROUP BY Date, Country) GROUP BY Country));"
            
            if print_sql:
                print(sql)
            cur = con.execute(sql)
            ans = cur.fetchone()
            print(ans[0])
            return

def generate_result(query, print_sql):
    if query['Operation Type']=='cases':
        generate_case_query_result(query,print_sql)
        if query['Place']['no_match']:
            print("No result found for: ", query['Place']['no_match'])
    else:
        generate_place_query_result(query,print_sql)

In [182]:
for query in queries:
    print(query['query'])
    generate_result(query,print_sql_queries)
    print()

State having minimum number of death ratio till between may and august.
SELECT Province_State FROM (SELECT Province_State, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM(Deaths) as sum FROM timeseries WHERE Province_State <> '' AND Date BETWEEN '2020-05-31' AND '2020-08-01' GROUP BY Date, Province_State) GROUP BY Province_State) WHERE cases = (SELECT MIN(cases) from (SELECT Province_State, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM(Deaths) as sum FROM timeseries WHERE Province_State <> '' AND Date BETWEEN '2020-05-31' AND '2020-08-01' GROUP BY Date, Province_State) GROUP BY Province_State));
Anguilla

State having maximum number of active cases in the world?
SELECT Province_State FROM (SELECT Province_State, (MAX(sum)-MIN(sum)) as cases FROM (SELECT Date, Province_State, SUM(Confirmed-Recovered-Deaths) as sum FROM timeseries WHERE Province_State <> '' AND Date BETWEEN '2020-01-01' AND '2020-12-31' GROUP BY Date, Province_State) GROUP BY Pro

In [183]:
con.close()